# Beltway - Analyzing How Remapping Interacts with map - 
## Where changes occur, stability of map, etc 
## Feb 14 2021 WH

### Imports

In [28]:
import ratterdam_CoreDataStructures as Core
import ratterdam_ParseBehavior as Parse
import numpy as np
import utility_fx as util
import ratterdam_Defaults as Def
import ratterdam_DataFiltering as Filt
import pandas as pd
from collections import OrderedDict
import scipy.ndimage # use center_of_mass from here

In [2]:
%qtconsole --style paraiso-dark

### Load Day Population

In [163]:
rat = 'R859'
expCode = 'BRD5'
datafile = f'E:\\Ratterdam\\{rat}\\{rat}{expCode}\\'

qualThresh = 2
alleyTracking, alleyVisits,  txtVisits, p_sess, ts_sess = Parse.getDaysBehavioralData(datafile, expCode)
clustlist, clustQuals = util.getClustList(datafile) # clustList and clustQuals share same order. ith entry in each is the name and qual of same cell. 
population = OrderedDict()

# Load Data
for i,clust in enumerate(clustlist):
    if clustQuals[i] >= qualThresh:
        unit = Core.UnitData(clust, datafile, expCode, Def.alleyBounds, alleyVisits, txtVisits, p_sess, ts_sess)
        unit.loadData_raw()
        validalleys = []
        valid, acorr, alleys = util.checkInclusion(unit, 3) # 2nd arg to util.checkInclusion is how many comps made per alley. This 
                                                            # value (usually 3) is not being saved here and is defined in the relevant R code so ignore it here
        if valid:
            print(clust)
            unit.acorr = acorr
            unit.validAlleys = alleys
            population[clust] = unit

E:\UserData\Documents\GitHub\ratterdam\Beltway_Project\ratterdam_CoreDataStructures.py:178: RuntimeWarning: divide by zero encountered in reciprocal
  n = (hs*np.reciprocal(ho))*30
E:\UserData\Documents\GitHub\ratterdam\Beltway_Project\ratterdam_CoreDataStructures.py:178: RuntimeWarning: invalid value encountered in multiply
  n = (hs*np.reciprocal(ho))*30
E:\UserData\Documents\GitHub\ratterdam\Beltway_Project\ratterdam_CoreDataStructures.py:184: RuntimeWarning: divide by zero encountered in reciprocal
  n = (ls* np.reciprocal(lo)) * 30
E:\UserData\Documents\GitHub\ratterdam\Beltway_Project\ratterdam_CoreDataStructures.py:184: RuntimeWarning: invalid value encountered in multiply
  n = (ls* np.reciprocal(lo)) * 30
E:\UserData\Documents\GitHub\ratterdam\utility_fx.py:330: RuntimeWarning: invalid value encountered in true_divide
  Z=VV/WW
E:\UserData\Documents\GitHub\ratterdam\utility_fx.py:680: RuntimeWarning: Mean of empty slice
  mean = np.nanmean(rms, axis=0)
E:\UserData\Documents\Gi

TT1\cl-maze1.1
TT1\cl-maze1.4
TT1\cl-maze1.5
TT1\cl-maze1.6
TT10\cl-maze1.10
TT10\cl-maze1.11
TT10\cl-maze1.12
TT10\cl-maze1.6
TT10\cl-maze1.7
TT13\cl-maze1.1
TT13\cl-maze1.2
TT13\cl-maze1.3
TT13\cl-maze1.4
TT13\cl-maze1.6
TT13\cl-maze1.7
TT13\cl-maze1.8
TT6\cl-maze1.10
TT6\cl-maze1.11
TT6\cl-maze1.12
TT6\cl-maze1.14
TT6\cl-maze1.15
TT6\cl-maze1.16
TT6\cl-maze1.2
TT6\cl-maze1.3
TT6\cl-maze1.4
TT6\cl-maze1.6
TT6\cl-maze1.7
TT6\cl-maze1.8
TT6\cl-maze1.9


### Significant Units for the indicated days according to LMER run 2-13-21 
sigUnits= {'R781BRD3':
[
('TT3\\cl-maze1.1',6),
  ('TT3\\cl-maze1.2',6),
  ('TT3\\cl-maze1.8',6),
  ('TT5\\cl-maze1.1',7)],
 'R781BRD4': [
('TT3\\cl-maze1.1',9),
  ('TT3\\cl-maze1.6',9),
  ('TT6\\cl-maze1.7',9),
  ('TT9\\cl-maze1.1',9)],
 'R808BRD6': [
('TT1\\cl-maze1.1',5),
  ('TT5\\cl-maze1.5',9),
  ('TT6\\cl-maze1.1',4),
  ('TT6\\cl-maze1.2',3)],
 'R859BRD3': [
('TT13\\cl-maze1.2',1),
  ('TT6\\cl-maze1.2',8),
  ('TT7\\cl-maze1.3',1),
  ('TT8\\cl-maze1.1',8)],
 'R859BRD5': [
('TT1\\cl-maze1.6',4),
  ('TT10\\cl-maze1.6',1),
  ('TT13\\cl-maze1.1',7),
  ('TT13\\cl-maze1.4',8),
  ('TT6\\cl-maze1.3',1),
  ("TT6\\cl-maze1.9",9)]}

In [164]:
for name in population.keys():
    isSig = False
    matches = [entry[1] if entry[0] == name else False for entry in sigUnits[rat+expCode]]
    for m in matches:
        if m is not False:
            alley = m
            isSig = True
    if isSig:
            population[name].sigLMER = True
            population[name].sigAlley = alley
    else:
            population[name].sigLMER = False
            population[name].sigAlley = False
superPopulation[rat+expCode] = population

## Map Stability
### COM Shifts, rate diffs, Area between curve diffs

In [54]:
def calcPairDeltaCOM(x, y):
    """
    Input: x,y are (Def.singleAlleyBins[0]-1,) np array
            containing the session average 1d RM for textures
            x,y
    center of mass from scipy.ndimage is calculated on (nan masked)
    each vector and unsigned difference is returned. 
    """
    
    xmask = np.ma.array(x,mask=np.isnan(x))
    ymask = np.ma.array(y,mask=np.isnan(y))
    comx, comy = scipy.ndimage.center_of_mass(xmask)[0], scipy.ndimage.center_of_mass(ymask)[0]
    deltacom = np.abs(comx-comy)
    
    return deltacom


In [64]:
def calcMaxDeltaCOM(unit, alley):
    """
    """
    maxCOM = 0
    for pair in [['A','B'], ['B','C'],['A','C']]:
        rmx = unit.linRMS[alley][pair[0]]
        rmy = unit.linRMS[alley][pair[1]]
        dcom = calcPairDeltaCOM(rmx,rmy)
        if dcom > maxCOM:
            maxCOM = dcom
    #maxCOM is in bins, convert to cm
    maxCOM = maxCOM * Def.cmPerBin
    
    return maxCOM

In [71]:
def calcPairABC(x,y):
    """
    Input: x,y are (Def.singleAlleyBins[0]-1,) np array
            containing the session average 1d RM for textures
            x,y
            
    Area between curves (ABC) is found by taking the
    unsigned difference in the curves then the area
    by np.trapz. Returns ABC in cm. 
    """
    xmask = np.ma.array(x,mask=np.isnan(x))
    ymask = np.ma.array(y,mask=np.isnan(y))
    unsigndiff = np.abs(np.diff(xmask-ymask))
    area = np.trapz(unsigndiff, dx=1)
    
    return area

In [66]:
def calcMaxABC(unit, alley):
    maxABC = 0
    for pair in [['A','B'], ['B','C'],['A','C']]:
        rmx = unit.linRMS[alley][pair[0]]
        rmy = unit.linRMS[alley][pair[1]]
        abc = calcPairABC(rmx,rmy)
        if abc > maxABC:
            maxABC = abc
    #maxABC already in cm 
    
    return maxABC

In [109]:
def calcPairRateDiff(x,y):
    """
    Input: x,y are (Def.singleAlleyBins[0]-1,) np array
            containing the session average 1d RM for textures
            x,y
    Calculate diff in max rate between vectord
    """
    xmask = np.ma.array(x,mask=np.isnan(x))
    ymask = np.ma.array(y,mask=np.isnan(y))
    
    return np.abs(xmask.max()-ymask.max())

In [107]:
def calcMaxRateDiff(unit, alley):
    """
    Calc max rate diff between texture pairs
    for a cell at an alley and return it.
    rate diff is unsigned.
    """
    maxdiff = 0
    for pair in [['A','B'], ['B','C'],['A','C']]:
        rmx = unit.linRMS[alley][pair[0]]
        rmy = unit.linRMS[alley][pair[1]]
        rdiff = calcPairRateDiff(rmx,rmy)
        if rdiff > maxdiff:
            maxdiff = rdiff
    
    return maxdiff

In [201]:
for day in ['R781BRD3', 'R781BRD4', 'R808BRD6', 'R808BRD7', 'R859BRD3', 'R859BRD5']:
    for name,unit in superPopulation[day].items():
        for alley in unit.validAlleys:
            x, y = calcMaxABC(unit, alley), calcMaxRateDiff(unit, alley)
            if unit.sigLMER == True and unit.sigAlley == Def.beltwayAlleyLookup[alley]:
                color = 'r'
                plt.scatter(x, y, s=40, color=color,zorder=99)
            else:
                color = 'grey'
                plt.scatter(x, y, s=40, color=color,)
            
plt.tick_params(axis='both', labelsize=18)

plt.xlabel("Max Area Between Curves, a.u.", fontsize=22)
plt.ylabel("Max Rate Difference, Hz", fontsize=22)

Text(0, 0.5, 'Max Rate Difference, Hz')

## Analyzing Locations of Rate Remapping effect
### Note this will be for all cells, not just the ones that pass LMER

In [214]:
ncells = 0
for day in superPopulation.keys():
    ncells += len(superPopulation[day])
nbins = Def.singleAlleyBins[0]-1

In [404]:
maxdiffs = np.empty((0,nbins))
for day in superPopulation.keys():
    for unitname in superPopulation[day].keys():
        
        unit = superPopulation[day][unitname]
        for alley in unit.validAlleys:
            if unit.sigLMER == True and unit.sigAlley == Def.beltwayAlleyLookup[alley]:
            #if True:
                ab = np.abs(unit.linRMS[alley]['A']-unit.linRMS[alley]['B'])
                bc = np.abs(unit.linRMS[alley]['C']-unit.linRMS[alley]['B'])
                ac = np.abs(unit.linRMS[alley]['C']-unit.linRMS[alley]['A'])
                d = np.vstack((ab,bc,ac))
                md = np.nanmax(d,axis=0)
                
                # want to align alleys with entrance on L and exit on R
                if unit.sigAlley in [16, 11, 8, 10]:
                    md = np.flip(md)
                    
                maxdiffs = np.vstack((maxdiffs,md))
maxdiffs = np.nan_to_num(maxdiffs)

C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: All-NaN slice encountered
  del sys.path[0]


In [405]:
# make plot
maxdiffs = np.sort(maxdiffs,axis=0)
plt.imshow(maxdiffs,aspect='auto',interpolation='kaiser',cmap=cmap,vmax=20)
plt.tick_params(axis='both', labelsize=18)
plt.xlabel("Spatial Bin", fontsize=22)
plt.ylabel("Cell/Alley Condition", fontsize=22)
plt.vlines(23/2,0,20.5,color='green')
plt.vlines((23/2)-4.66,0,20.5,color='black')
plt.vlines((23/2)+4.66,0,20.5,color='black')

In [338]:
allcoms = []
for day in superPopulation.keys():
    for unitname in superPopulation[day].keys():
        unit = superPopulation[day][unitname]
        for alley in unit.validAlleys:
            avg = np.nanmean(np.vstack((unit.linRMS[alley]['A'],unit.linRMS[alley]['C'],unit.linRMS[alley]['C'])),axis=0)
            mask = np.ma.array(avg, mask=np.isnan(avg))
            com = scipy.ndimage.center_of_mass(mask)[0]
            allcoms.append(com)
allcoms = np.asarray(allcoms)

C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: Mean of empty slice
  
